In [1]:
# import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name())

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


########################### Load dataset ###########################

In [2]:
attribute_1 = "african"
attribute_2 = "european"
train_dataset = torch.load('2_train_'+attribute_1+'_'+attribute_2+'_tokens.pt')
val_dataset = torch.load('2_val_'+attribute_1+'_'+attribute_2+'_tokens.pt')
# test_dataset = torch.load('datasets/toy_test_attributes_datasets_seed_42_albert_large.pt')

In [3]:
# change model to albert

In [4]:
# dataset_size = len(dataset)
# random_split =  torch.utils.data.random_split
# test_size = int(0.1 * dataset_size)
# val_size = int(0.05 * dataset_size)
# train_size = dataset_size - (test_size + val_size)

# train_dataset, test_dataset, val_dataset = random_split(dataset,
#                                            [train_size, test_size, val_size])

In [5]:
# import albert_large_model
# import importlib
# importlib.reload(albert_large_model)
# AlbertForSequenceClassification = albert_large_model.AlbertForSequenceClassification

# model = AlbertForSequenceClassification.from_pretrained("albert-large-v2", num_labels = 2)
# # model = torch.nn.DataParallel(model, device_ids=None)
# model.cuda(1)

In [6]:
# import bert_large_model
# import importlib
# importlib.reload(bert_large_model)
# BertForSequenceClassification = bert_large_model.BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained("bert-large-uncased", state_dict=torch.load("../../../../kaneko_debiased_bert/context-debias-main/debiased_models/42/bert_large/checkpoint-best/pytorch_model.bin"), num_labels = 2)

# # Tell pytorch to run this model on the GPU.
# model.cuda()

In [7]:
import bert_model
import importlib
# importlib.reload(albert_large_model)
BertForMaskedLM = bert_model.BertForMaskedLM
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
# model = AlbertForSequenceClassification.from_pretrained("albert-large-v2", num_labels = 2, state_dict=torch.load("../../../../kaneko_debiased_bert/context-debias-main/debiased_models/42/albert_large/checkpoint-best/pytorch_model.bin"))

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly ini

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [8]:
# # model3 = torch.load("models/liang_BERT_classifier_model.pth")
# import liang_albert_model
# import importlib
# importlib.reload(liang_bert_model)
# BertForSequenceClassification = liang_bert_model.BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)
# # model.bert.load_state_dict(torch.load("attribute_models/pretrained_BERT_classifier_model.pth").bert.state_dict())
# model.cuda(1)

In [9]:
# import liang_albert_model
# import importlib
# importlib.reload(liang_albert_model)
# BertForSequenceClassification = liang_bert_model.BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)
# model.cuda(1)

In [10]:
# import bert_model
# import importlib
# importlib.reload(bert_model)
# BertForSequenceClassification = bert_model.BertForSequenceClassification

# import torch
# # Load BertForSequenceClassification, the pretrained BERT model with a single 
# # linear classification layer on top. 
# model_cda_bert_encoder = torch.load("../../../CDA/counterfactual-data-substitution-master/CDA_model.pth").bert.encoder
# # model = BertModel.from_pretrained('bert-base-uncased')




# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)
# model.bert.encoder.load_state_dict(model_cda_bert_encoder.state_dict())

# # Tell pytorch to run this model on the GPU.
# model.cuda()

########################### optimization, training, and validation functions ###########################


In [11]:

import time
import datetime

from transformers.optimization import AdamW
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


from transformers.optimization import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 10

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)




def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# import numpy as np


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    # print(preds)
    # pred_flat = np.argmax(preds, axis=1).flatten()
    pred_flat = torch.round((torch.sign(preds) + 1) / 2)
    pred_flat = np.array(pred_flat.squeeze().long().tolist())
    labels_flat = np.array(labels.squeeze().tolist())
    # print("pred", pred_flat,"\n")
    # print("labl", labels_flat,"\n")
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import random
from tqdm.notebook import tqdm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
def train(model, optimizer, losslogger, start_epoch, epochs, run_id, train_dataloader, validation_dataloader, checkpoint_name):
    # Set the seed value all over the place to make this reproducible.
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    t = tqdm(range(start_epoch,epochs))

    for epoch_i in t:

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

                # if step % 1200 == 0 and not step == 0:
                #             torch.save(self.model_F, "classification_model_try_"+str(epoch_i)+".pth")

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            stereotype_token_index = batch[3].to(device)
            b_target_labels = batch[4].to(device)
            
#             #     convert input ids to one hot encoding
#             b_input_ids = b_input_ids.reshape(len(b_input_ids), seq_len, 1)
#             y_onehot = torch.FloatTensor(len(b_input_ids), seq_len, vocab_length)
#             y_onehot.zero_()
#             y_onehot.scatter_(2, b_input_ids, 1)
#             y_onehot = y_onehot.float().to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            outputs = model(b_input_ids, 
                                 token_type_ids=None,
                                 stereotype_token_index=stereotype_token_index,
                                 attention_mask=b_input_mask, 
                                 labels=b_target_labels,
                                )

            loss, logits = outputs.loss, outputs.logits
            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
#             print(outputs)
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        if epoch_i%1==0:
            torch.save(model, "classification_model_"+attribute_1+"_"+attribute_2+"_"+str(epoch_i)+".pth")
        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)
        
        # set in logs
        df = pd.DataFrame()
        df['chackpoint_name'] = pd.Series(checkpoint_name)
        df['epoch'] = pd.Series(epoch_i)
        df['Loss'] = pd.Series(loss.data.item())
        df['run'] = run_id
        # losslogger = losslogger.append(df)
        
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
        state = {'epoch': epoch_i + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'losslogger': losslogger, }
#         torch.save(state, f'{checkpoint_name}')

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            stereotype_token_index = batch[3].to(device)
            b_target_labels = batch[4].to(device)
            

            
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                outputs = model(b_input_ids, 
                                     token_type_ids=None,
                                     stereotype_token_index=stereotype_token_index,
                                     attention_mask=b_input_mask, 
                                     labels=b_target_labels,
                                    )
            
            loss, logits = outputs.loss, outputs.logits
            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.reshape(len(b_target_labels)).detach().cpu()
#             print(logits.shape)
            label_ids = b_target_labels.to('cpu')
#             print(label_ids.shape)

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)
#             print(total_eval_accuracy)


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
#         print(total_eval_accuracy)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

#     print("")
#     print("Training complete!")

#     print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
          
def load_checkpoint(model, optimizer, losslogger, filename):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        losslogger = checkpoint['losslogger']
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch, losslogger          
          
          
          
          


/opt/conda/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


########################### Start training with checkpoint ###########################

In [12]:
# first training epoch
# Start
start_epoch = 0

# Logger
losslogger = pd.DataFrame()

# Checkpoint name
checkpoint_name = 'checkpoint.pth.tar'

train(model, optimizer, losslogger, start_epoch, epochs, 0, train_dataloader, validation_dataloader, checkpoint_name)
time.sleep(8)

  0%|          | 0/10 [00:00<?, ?it/s]


======== Epoch 1 / 10 ========
Training...
  Batch    40  of  2,779.    Elapsed: 0:00:04.
  Batch    80  of  2,779.    Elapsed: 0:00:07.
  Batch   120  of  2,779.    Elapsed: 0:00:09.
  Batch   160  of  2,779.    Elapsed: 0:00:12.
  Batch   200  of  2,779.    Elapsed: 0:00:15.
  Batch   240  of  2,779.    Elapsed: 0:00:18.
  Batch   280  of  2,779.    Elapsed: 0:00:21.
  Batch   320  of  2,779.    Elapsed: 0:00:24.
  Batch   360  of  2,779.    Elapsed: 0:00:27.
  Batch   400  of  2,779.    Elapsed: 0:00:30.
  Batch   440  of  2,779.    Elapsed: 0:00:33.
  Batch   480  of  2,779.    Elapsed: 0:00:36.
  Batch   520  of  2,779.    Elapsed: 0:00:38.
  Batch   560  of  2,779.    Elapsed: 0:00:41.
  Batch   600  of  2,779.    Elapsed: 0:00:44.
  Batch   640  of  2,779.    Elapsed: 0:00:47.
  Batch   680  of  2,779.    Elapsed: 0:00:50.
  Batch   720  of  2,779.    Elapsed: 0:00:53.
  Batch   760  of  2,779.    Elapsed: 0:00:56.
  Batch   800  of  2,779.    Elapsed: 0:00:59.
  Batch   840  o

In [13]:
# torch.save(model, "bert_pretrained_attribute_classifier_model.pth")

In [14]:
# cgcgcgcg

In [15]:

# import time
# import datetime

# from transformers.optimization import AdamW
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# # The DataLoader needs to know our batch size for training, so we specify it 
# # here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# # size of 16 or 32.
# batch_size = 32





# from transformers.optimization import get_linear_schedule_with_warmup

# # Number of training epochs. The BERT authors recommend between 2 and 4. 
# # We chose to run for 4, but we'll see later that this may be over-fitting the
# # training data.
# # epochs = 4

# # Total number of training steps is [number of batches] x [number of epochs]. 
# # (Note that this is not the same as the number of training samples).
# # total_steps = len(test_dataloader) * epochs





# def format_time(elapsed):
#     '''
#     Takes a time in seconds and returns a string hh:mm:ss
#     '''
#     # Round to the nearest second.
#     elapsed_rounded = int(round((elapsed)))
    
#     # Format as hh:mm:ss
#     return str(datetime.timedelta(seconds=elapsed_rounded))

# # import numpy as np


# # Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#     pred_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels.flatten()
#     return np.sum(pred_flat == labels_flat) / len(labels_flat)


# import random
# from tqdm.notebook import tqdm

# # This training code is based on the `run_glue.py` script here:
# # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# def train(model, losslogger, run_id, test_dataloader):
#     seed_val = 42

#     random.seed(seed_val)
#     np.random.seed(seed_val)
#     torch.manual_seed(seed_val)
#     torch.cuda.manual_seed_all(seed_val)

#     # We'll store a number of quantities such as training and validation loss, 
#     # validation accuracy, and timings.
#     training_stats = []

#     # Measure the total training time for the whole run.
#     total_t0 = time.time()

#     # For each epoch...
# #     t = tqdm(range(start_epoch,epochs))

# #     for epoch_i in t:

   

#     # ========================================
#     #               Validation
#     # ========================================
#     # After the completion of each training epoch, measure our performance on
#     # our validation set.

#     print("")
#     print("Running Validation...")

#     t0 = time.time()

#     # Put the model in evaluation mode--the dropout layers behave differently
#     # during evaluation.
#     model.eval()

#     # Tracking variables 
#     total_eval_accuracy = 0
#     total_eval_loss = 0
#     nb_eval_steps = 0

#     # Evaluate data for one epoch
#     for batch in test_dataloader:

#         # Unpack this training batch from our dataloader. 
#         #
#         # As we unpack the batch, we'll also copy each tensor to the GPU using 
#         # the `to` method.
#         #
#         # `batch` contains three pytorch tensors:
#         #   [0]: input ids 
#         #   [1]: attention masks
#         #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         stereotype_token_index = batch[3].to(device)
#         b_target_labels = batch[4].to(device)



#         # Tell pytorch not to bother with constructing the compute graph during
#         # the forward pass, since this is only needed for backprop (training).
#         with torch.no_grad():        

#             # Forward pass, calculate logit predictions.
#             # token_type_ids is the same as the "segment ids", which 
#             # differentiates sentence 1 and 2 in 2-sentence tasks.
#             # The documentation for this `model` function is here: 
#             # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
#             # Get the "logits" output by the model. The "logits" are the output
#             # values prior to applying an activation function like the softmax.
#             outputs = model(b_input_ids, 
#                                  token_type_ids=None,
#                                  stereotype_token_index=stereotype_token_index,
#                                  attention_mask=b_input_mask, 
#                                  labels=b_target_labels,
#                                 )

#         loss, logits = outputs.loss, outputs.logits
#         # Accumulate the validation loss.
#         total_eval_loss += loss.item()

#         # Move logits and labels to CPU
#         logits = logits.reshape(len(b_target_labels),2).detach().cpu().numpy()
# #             print(logits.shape)
#         label_ids = b_target_labels.to('cpu').numpy()
# #             print(label_ids.shape)

#         # Calculate the accuracy for this batch of test sentences, and
#         # accumulate it over all batches.
#         total_eval_accuracy += flat_accuracy(logits, label_ids)
# #             print(total_eval_accuracy)


#     # Report the final accuracy for this validation run.
#     avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
# #         print(total_eval_accuracy)
#     print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

#     # Calculate the average loss over all of the batches.
#     avg_val_loss = total_eval_loss / len(test_dataloader)

#     # Measure how long the validation run took.
#     validation_time = format_time(time.time() - t0)

#     print("  Validation Loss: {0:.2f}".format(avg_val_loss))
#     print("  Validation took: {:}".format(validation_time))
test_dataloader = DataLoader(
#         test_dataset, # The validation samples.
#         sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#         batch_size = batch_size # Evaluate with this batch size.
#         )

# # Start
# start_epoch = 0

# # Logger
# losslogger = pd.DataFrame()

# # Checkpoint name
# checkpoint_name = 'checkpoint.pth.tar'

# train(model, losslogger, 0, test_dataloader)
# time.sleep(8)
# #     # Record all statistics from this epoch.
# #     training_stats.append(
# #         {
# #             'Training Loss': avg_train_loss,
# #             'Valid. Loss': avg_val_loss,
# #             'Valid. Accur.': avg_val_accuracy,
# #             'Training Time': training_time,
# #             'Validation Time': validation_time
# #         }
# #     )

          


SyntaxError: incomplete input (4031181904.py, line 192)

In [ ]:
# test_dataloader = DataLoader(
#         test_dataset, # The validation samples.
#         sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#         batch_size = batch_size # Evaluate with this batch size.
#         )

# # Start
# start_epoch = 0

# # Logger
# losslogger = pd.DataFrame()

# # Checkpoint name
# checkpoint_name = 'checkpoint.pth.tar'

# train(model, losslogger, 0, test_dataloader)
# time.sleep(8)

In [ ]:
# torch.save(model, "attribute_models/liang_BERT_classifier_model.pth")

########################### Continue training from checkpoint ###########################

In [ ]:
# epochs = 4
# # Load and continute train // run 1
# # Checkpoint name
# checkpoint_name = 'checkpoint.pth.tar'
# losslogger = pd.DataFrame()
# model, optimizer, start_epoch, losslogger = load_checkpoint(model, optimizer, losslogger, filename=checkpoint_name)
# # !rm 'checkpoint.pth.tar'
# train(model, optimizer, losslogger, start_epoch, epochs, 1, train_dataloader, validation_dataloader, checkpoint_name)
# time.sleep(8)
